# <div align="center">Predict Car Traffic Injury</div>



<br>
<br>

In this document we follow the CRISP-DM process and methodolgies to develop our project.
Full details are described in the report document of the project.

In the file, we will do:
* Coding + Analysis for results.
* Code insights.
* Apply the practices learned for the business part, including understanding domain concepts and details about each feature.


<br>
<br>
<br>
<br>


**How to read this document**

1. Text in black (or White, depends on reader theme background) - Describes section or sub-section titles and details.

<span style="color:cyan">

2. Text in "cyan" is analysis realted to previous code section.
</span>


<br>
<br>
<br>
<br>


# 1. CRISP-DM: Business Understanding

1.1. Background and Objective<br>
1.2. Descriptive Features and Domain Concepts<br>
1.3. Domain Concepts Explained<br>
1.4. Relevant Resources<br>

<br>
<br>

<span style="color:cyan">

# 1.1. Background and Objective<br>
Traffic Accidents globally rank high in causing deaths and injuries. Traffic accidents have significant health care and economic impact.<br>
According to World Health Organization traffic accidents is one of the leading death causes worldwide.<br>
<br>
In this project, we are utilizing advanced machine learning models and data analytics to forecast accident severity for strategic planning and response, aiming to reduce traffic-related injuries, fatalities, and economic costs with targeted preventive measures and efficient resource allocation.<br>
<br>
We addressing the gap in predictive knowledge of traffic accident severity factors in Chicago's urban landscape to enhancing public health and safety by identifying risk patterns and contributing factors through historical data analysis.<br>

</span>

<br>
<br>
<br>


<span style="color:cyan">

## 1.2. Descriptive Features and Domain Concepts
<br>
<br>

| Column Name                    | Type           | Short Description                                                                                                                         | Domain Concept           |
|--------------------------------|----------------|-------------------------------------------------------------------------------------------------------------------------------------------|-------------------------|
| CRASH_RECORD_ID                | Text           | Unique identifier for the crash record, linking to the same crash in the Vehicles and People datasets.                                    | Identification          |
| CRASH_DATE_EST_I               | Text           | Indicates if the crash date was estimated by a desk officer or reporting party.                                                           | Crash Details           |
| CRASH_DATE                     | DateTime       | Date and time of the crash as entered by the reporting officer.                                                                           | Crash Details           |
| POSTED_SPEED_LIMIT             | Number         | Posted speed limit as determined by the reporting officer.                                                                                | Environmental Factors   |
| TRAFFIC_CONTROL_DEVICE         | Text           | Traffic control device present at the crash location as determined by the reporting officer.                                              | Environmental Factors   |
| DEVICE_CONDITION               | Text           | Condition of the traffic control device as determined by the reporting officer.                                                           | Environmental Factors   |
| WEATHER_CONDITION              | Text           | Weather condition at the time of the crash as determined by the reporting officer.                                                        | Environmental Factors   |
| LIGHTING_CONDITION             | Text           | Lighting condition at the time of the crash as determined by the reporting officer.                                                       | Environmental Factors   |
| FIRST_CRASH_TYPE               | Text           | Type of first collision in the crash.                                                                                                     | Crash Details           |
| TRAFFICWAY_TYPE                | Text           | Trafficway type as determined by the reporting officer.                                                                                   | Environmental Factors   |
| LANE_CNT                       | Number         | Total number of through lanes in either direction, excluding turn lanes, at the crash location as determined by the reporting officer.     | Environmental Factors   |
| ALIGNMENT                      | Text           | Street alignment at the crash location as determined by the reporting officer.                                                            | Environmental Factors   |
| ROADWAY_SURFACE_COND           | Text           | Road surface condition at the crash location as determined by the reporting officer.                                                      | Environmental Factors   |
| ROAD_DEFECT                    | Text           | Road defects present at the crash location as determined by the reporting officer.                                                        | Environmental Factors   |
| REPORT_TYPE                    | Text           | Administrative report type of the crash.                                                                                                   | Documentation          |
| CRASH_TYPE                     | Text           | General severity classification for the crash.                                                                                             | Crash Details           |
| INTERSECTION_RELATED_I         | Text           | Indicates if an intersection played a role in the crash as observed by the police officer.                                                | Environmental Factors   |
| NOT_RIGHT_OF_WAY_I             | Text           | Indicates if the crash began or first contact was made outside of the public right-of-way.                                                | Legal                   |
| HIT_AND_RUN_I                  | Text           | Indicates if the crash involved a driver who fled the scene without exchanging information and/or rendering aid.                          | Legal                   |
| DAMAGE                         | Text           | Estimated damage from the crash as observed in the field.                                                                                 | Crash Outcome           |
| DATE_POLICE_NOTIFIED           | DateTime       | Calendar date on which the police were notified of the crash.                                                                             | Documentation          |
| PRIM_CONTRIBUTORY_CAUSE        | Text           | Primary factor contributing to the cause of the crash as determined by officer judgment.                                                  | Crash Analysis         |
| SEC_CONTRIBUTORY_CAUSE         | Text           | Secondary factor contributing to the cause of the crash as determined by officer judgment.                                                | Crash Analysis         |
| STREET_NO                      | Number         | Street address number of the crash location as determined by the reporting officer.                                                       | Location Details       |
| STREET_DIRECTION               | Text           | Street address direction (N, E, S, W) of the crash location as determined by the reporting officer.                                       | Location Details       |
| STREET_NAME                    | Text           | Street name of the crash location as determined by the reporting officer.                                                                 | Location Details       |
| BEAT_OF_OCCURRENCE             | Number         | Chicago Police Department beat ID where the crash occurred.                                                                               | Administrative         |
| PHOTOS_TAKEN_I                 | Text           | Indicates if photos were taken at the crash location by the Chicago Police Department.                                                    | Documentation          |
| STATEMENTS_TAKEN_I             | Text           | Indicates if statements were taken from units involved in the crash.                                                                      | Documentation          |
| DOORING_I                      | Text           | Indicates if the crash involved a vehicle occupant opening a door into the path of a bicyclist.                                           | Crash Type             |
| WORK_ZONE_I                    | Text           | Indicates if the crash occurred in an active work zone.                                                                                   | Environmental Factors   |
| WORK_ZONE_TYPE                 | Text           | Type of work zone, if any, where the crash occurred.                                                                                      | Environmental Factors   |
| WORKERS_PRESENT_I              | Text           | Indicates if construction workers were present in the work zone at the crash location.                                                    | Environmental Factors   |
| NUM_UNITS                      | Number         | Number of units involved in the crash, representing different modes of traffic with independent trajectories.                             | Crash Details           |
| MOST_SEVERE_INJURY             | Text           | Most severe injury sustained by any person involved in the crash.                                                                         | Injury Analysis        |
| INJURIES_TOTAL                 | Number         | Total number of people sustaining fatal, incapacitating, non-incapacitating, and possible injuries as determined by the reporting officer. | Injury Analysis        |
| INJURIES_FATAL                 | Number         | Total number of people sustaining fatal injuries in the crash.                                                                            | Injury Analysis        |
| INJURIES_INCAPACITATING        | Number         | Total number of people sustaining incapacitating/serious injuries in the crash.                                                           | Injury Analysis        |
| INJURIES_NON_INCAPACITATING    | Number         | Total number of people sustaining non-incapacitating injuries in the crash.                                                               | Injury Analysis        |
| INJURIES_REPORTED_NOT_EVIDENT  | Number         | Total number of people sustaining possible injuries in the crash as determined by the reporting officer.                                  | Injury Analysis        |
| INJURIES_NO_INDICATION         | Number         | Total number of people with no injuries in the crash as determined by the reporting officer.                                              | Injury Analysis        |
| INJURIES_UNKNOWN               | Number         | Total number of people for whom the injury status, if any, is unknown.                                                                    | Injury Analysis        |
| CRASH_HOUR                     | Number         | Hour of the day when the crash occurred, derived from CRASH_DATE.                                                                         | Time Details           |
| CRASH_DAY_OF_WEEK              | Number         | Day of the week when the crash occurred, derived from CRASH_DATE. Sunday=1                                                                | Time Details           |
| CRASH_MONTH                    | Number         | Month when the crash occurred, derived from CRASH_DATE.                                                                                   | Time Details           |
| LATITUDE                       | Number         | Latitude of the crash location as determined by the reporting officer.                                                                    | Geographic Information |
| LONGITUDE                      | Number         | Longitude of the crash location as determined by the reporting officer.                                                                   | Geographic Information |
| LOCATION                       | Point          | Geographic location of the crash as determined by the reporting officer, allowing for mapping and geographic analysis.                    | Geographic Information |


<br>
<br>
<br>

</span>


<span style="color:cyan">



## 1.3. Domain Concepts Explained<br>


##### **1. Identification**
- **Description**: Features that uniquely identify the crash records or events, often used for linking data across different datasets.
- **Example**: 
  - **CRASH_RECORD_ID**: Serves as a unique identifier for each crash event, enabling connections with related records in vehicles and people datasets.

##### **2. Crash Details**
- **Description**: Specific details about the crash event, including the type of crash, date, and time, providing basic information on how and when the crash occurred.
- **Example**: 
  - **CRASH_DATE**: Provides the exact date and time the crash happened.
  - **FIRST_CRASH_TYPE**: Describes the initial type of collision.

##### **3. Environmental Factors**
- **Description**: Conditions surrounding the crash, such as weather, lighting, and road conditions, which can influence the occurrence and severity of crashes.
- **Example**: 
  - **WEATHER_CONDITION** and **LIGHTING_CONDITION**: Detail the environmental state at the time of the crash, affecting driving conditions and visibility.

##### **4. Legal**
- **Description**: Features related to legal aspects of the crash, including compliance with right-of-way rules and hit-and-run incidents.
- **Example**: 
  - **HIT_AND_RUN_I**: Indicates whether a driver involved in the crash fled the scene without providing information or rendering aid.

##### **5. Crash Outcome**
- **Description**: Descriptions of the immediate consequences of the crash, primarily in terms of physical damage.
- **Example**: 
  - **DAMAGE**: Estimates the monetary damage to property resulting from the crash.

##### **6. Documentation**
- **Description**: Administrative details about the crash reporting and documentation process, including whether photos or statements were taken.
- **Example**: 
  - **DATE_POLICE_NOTIFIED**: Records when the police were officially notified about the crash.

##### **7. Crash Analysis**
- **Description**: Factors identified as contributing to the crash, including primary and secondary causes as determined by officer judgment.
- **Example**: 
  - **PRIM_CONTRIBUTORY_CAUSE** and **SEC_CONTRIBUTORY_CAUSE**: Identify the main reasons behind the crash according to the reporting officer.

##### **8. Location Details**
- **Description**: Specifics about where the crash occurred, including street names and numbers, helping in pinpointing the exact crash location.
- **Example**: 
  - **STREET_NAME**, **STREET_NO**, and **STREET_DIRECTION**: Provide a detailed description of the crash site.

##### **9. Administrative**
- **Description**: Information related to police and emergency response jurisdictions, such as police beats.
- **Example**: 
  - **BEAT_OF_OCCURRENCE**: Refers to the Chicago Police Department beat where the crash took place.

##### **10. Crash Type**
- **Description**: Categories that describe specific scenarios or types of crashes, including incidents involving bicycles or work zones.
- **Example**: 
  - **DOORING_I**: Specifies if the crash involved a vehicle door being opened in the path of a bicyclist.

##### **11. Injury Analysis**
- **Description**: Detailed records of injuries resulting from the crash, categorizing the severity and type of injuries sustained.
- **Example**: 
  - **INJURIES_TOTAL**: Counts the total number of injuries.
  - **INJURIES_FATAL**: Counts the number of fatal injuries.

##### **12. Time Details**
- **Description**: Temporal information about the crash, including the hour, day of the week, and month when the crash occurred.
- **Example**: 
  - **CRASH_HOUR**, **CRASH_DAY_OF_WEEK**, and **CRASH_MONTH**: Provide insights into the timing patterns of crashes.

##### **13. Geographic Information**
- **Description**: Geospatial data related to the crash location, enabling mapping and location-based analysis.
- **Example**: 
  - **LATITUDE** and **LONGITUDE**: Give precise coordinates of the crash site.
  - **LOCATION**: Offers a mappable point of the incident.

Each domain concept helps categorize the dataset's features for easier analysis, highlighting different factors and details that contribute to a comprehensive understanding of traffic crashes.


<br>
<br>
<br>

</span>

<span style="color:cyan">

## 1.4. Relevant Resources

1. Dataset acquired from USA DATA GOV<br>
URL: https://catalog.data.gov/dataset/traffic-crashes-crashes

<br>

2. Full details about this section, please refer to the report document.


</span>

<br>
<br>
<br>

# 2. CRISP-DM: Data Understanding

In this section, the following will be addressed:<br><br>
**2.1. Collect Initial Data**<br>
2.1.1. Import the relevant Python packages that are going to be used.<br>
2.1.2. Acquire the data.<br>
2.1.3. Record of the data acquisition process.<br>
<br>
<br>

**2.2. Data Description**<br>
2.2.1. Shape of the Dataset.<br>
2.2.2. Head snippet.<br>
2.2.3. Dataset info.<br>
2.2.4. Numerical Feature Distribution.<br>
2.2.5. Catagorical Feature Distribution.<br>
2.2.6. HTML Reprot.<br>

<br>
<br>

**2.3. Explore the Data**<br>
2.3.1. Continuous Features Tabular Report<br>
2.3.2. Categorical Features Tabular Report <br>
2.3.3. Correlations Matrix<br>
2.3.4. HeatMap<br>
<br>
<br>

**2.4. Verify Data Quality**<br>
2.4.1. Missing Values Summary<br>
2.4.2. Irregular cardinality<br>
2.4.3. Outliers<br>
2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data.<br>


<br>
<br>

## 2.1. Collect Initial Data


This a generic intial step, to start working on the data:<br>
2.1.1. Import the relevant Python packages that are going to be used.<br>
2.1.2. Acquire the data.<br>
2.1.3. Record of the data acquisition process.<br>

In [ ]:
# 2.1.1. Import the relevant Python packages that are going to be used

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')


In [ ]:
# 2.1.2. Acquire the data

# A generic utilies file with generic functionallity
from misc.utilities import acquire_dataset

# Download & load the dataset
# If the file already downloaded, it will not be re-downloaded (to speed up work efficiency)
df = acquire_dataset()
# df

<span style="color:cyan">


2.1.2. Acquire the data<br>

The data acquired from: https://catalog.data.gov/dataset/traffic-crashes-crashes
<br>
The above website belongs to USA government free datasets advised by the course instructions.

<br>
<br>
<br>

</span>


In [ ]:
# 2.1.3. Helper functions

def get_continuous_feature_details(my_df, my_feature):
    my_df[my_feature] = pd.to_numeric(my_df[my_feature], errors='coerce')
    # Printing unique values
    unique_values = my_df[my_feature].unique()
    sorted_unique_values = np.sort(unique_values)
    len_unique_values = len(unique_values)
    unique_values_str = ', '.join(map(str, sorted_unique_values[:5])) + ', ..., ' + ', '.join(map(str, sorted_unique_values[-5:]))
    max_value = my_df[my_feature].max()
    min_value = my_df[my_feature].min()
    # print(f"{my_feature}: Unique Len={len_unique_values}, Unique={sorted_unique_values}, Max={max_value}, Min={min_value}")
    print(f"{my_feature}:\n  Unique Len={len_unique_values}\n  Unique={unique_values_str}\n  Max={max_value}\n  Min={min_value}")

    return len_unique_values, sorted_unique_values, max_value, min_value
    

def dump_feature_frequency_to_a_file(my_df, my_feature):
    value_counts = my_df[my_feature].value_counts()
    # value_counts.to_csv(f"{my_feature}_value_counts.csv", index=True, header=['Frequency'])
    # Open a file to write
    with open(f"{my_feature}_value_counts.txt", 'w') as file:
        # Write a header row
        file.write(f"{'Value':<50}       {'Frequency':<10}\n")
        file.write(f"{'-'*50}       {'-'*10}\n")
        
        # Iterate over the Series and write each value and its frequency
        for value, count in value_counts.items():
            file.write(f"{value:<50}       {count:<10}\n")


def dump_feature_classes_to_a_file(my_df, my_feature):
    # Extract unique values from the feature column
    unique_classes = my_df[my_feature].unique()
    # Sort the unique values if needed
    unique_classes_sorted = sorted(unique_classes)
    # Write the unique classes to a file
    with open(f"{my_feature}_unique_classes.txt", 'w') as file:
        for cls in unique_classes_sorted:
            file.write(cls + '\n')


<span style="color:cyan">


2.1.3. Helper functions<br>

Helper functions used periodically.

<br>
<br>
<br>

</span>

<br>
<br>

## 2.2. Data Description

In this phase we investigate the following aspects:<br>
2.2.1. Shape of the Dataset.<br>
2.2.2. Head snippet.<br>
2.2.3. Dataset info.<br>
2.2.4. Numerical Feature Distribution.<br>
2.2.5. Catagorical Feature Distribution.<br>
2.2.6 HTML report<br>

In [ ]:
# 2.2.1. Shape of the Dataset.

def get_dataset_shape(df):
    # Create a new DataFrame to display the shape information
    shape_df = pd.DataFrame({
        'Aspect': ['Records (Instances)', 'Features (Columns)'],
        'Number': [df.shape[0], df.shape[1]]
    })
    return shape_df

# Display the DataFrame
get_dataset_shape(df)


In [ ]:
# 2.2.2. Head snippet

def get_dataset_head(df):
    return df.head()

get_dataset_head(df)


In [ ]:
# 2.2.3. Dataset info

def get_dataset_info(df):
    df.info()

get_dataset_info(df)


In [ ]:
# 2.2.4. Numerical Feature Distribution

# Function for visualizaiotn of categorical feature distribution
def Numerical_feature_Dist(df):
    # Select only numeric columns for distribution plots
    numeric_df = df.select_dtypes(include=[np.number])
    # Determine the number of rows/columns for the subplot grid
    num_features = numeric_df.shape[1]
    num_rows = int(np.ceil(num_features / 3))  # Adjust the denominator to change the number of columns
    # Create a figure and a grid of subplots
    plt.figure(figsize=(15, num_rows * 5))
    for i, column in enumerate(numeric_df.columns):
        plt.subplot(num_rows, 3, i + 1)
        sns.histplot(numeric_df[column], kde=True, stat = 'density')
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
    # Adjust layout for better visualization
    plt.tight_layout()
    plt.show()

# Numerical_feature_Dist(df)

In [ ]:
## 2.2.5 Catagorical Feature Distribution


# Function for visualizaiotn of categorical feature distribution
def Categorical_feature_Dist(df):
    # Select only categorical columns
    categorical_df = df.select_dtypes(include=['object', 'category']) #exclude=['number']
    # Delete some of the columns due ot cardinality issues (they have alot of levels like Id)
    categorical_df = categorical_df.drop(['PRIM_CONTRIBUTORY_CAUSE','SEC_CONTRIBUTORY_CAUSE','STREET_NAME','DATE_POLICE_NOTIFIED','LOCATION','CRASH_RECORD_ID' , 'CRASH_DATE_EST_I','CRASH_DATE'] , axis = 1)
    # Determine the number of rows/columns for the subplot grid
    num_features = categorical_df.shape[1]
    num_rows = int(np.ceil(num_features / 3))  # Adjust for desired number of columns per row
    plt.figure(figsize=(15, num_rows * 5))
    for i, column in enumerate(categorical_df.columns):
        plt.subplot(num_rows, 3, i + 1)
        ax = sns.countplot(y=categorical_df[column])
        total = len(categorical_df[column])  # Total number of data points for the percentage calculation
        for p in ax.patches:
            percentage = '{:.1f}%'.format(100 * p.get_width()/total)
            x = p.get_x() + p.get_width() + 0.02  # Shifts the text to the right side of the bars
            y = p.get_y() + p.get_height()/2
            ax.annotate(percentage, (x, y))
        
        plt.title(f'Distribution of {column}')
        plt.xlabel('Count')
        plt.ylabel(column)

    plt.tight_layout()
    plt.show()

# Categorical_feature_Dist(df)


In [ ]:
# 2.2.6. HTML report

# Disable since it is taking high CPU/Memory resources
# profile = ProfileReport(df, title="Pandas Profiling Report",explorative=True)
# profile.to_file("pandas_profiling_report.html")

<span style="color:cyan">

##### 2.2. Data Description
##### Analysis

For all 2.2 section.

1. **Determining the Size of the Dataset**:
   - our dataset has a considerable size with **814,788 instances** and **51 features** that provide detailed attributes or properties of each instance.

2. **Previewing the Data**:
   - We have displayed the first few entries of the dataset to get an initial understanding of the data's structure and content. This is like reading the first page of a book to get a sense of the story.

3. **Assessing Data Completeness**:
   - The dataset's informational output revealed that certain features have missing values. For example, 'LANE_CNT' has many missing values, whereas 'CRASH_RECORD_ID' is complete. This insight is crucial for understanding which features are fully observed and which may require data imputation or cleaning.

4. **Visualizing Feature Distributions**:
   - We have created a series of plots to visualize the distribution of various features. These plots help us understand the frequency and pattern of different attributes. For example, the 'POSTED_SPEED_LIMIT' plot suggests most data points cluster around specific speed limits (30), while the 'LANE_CNT' plot indicates a right-skewed distribution, meaning lower lane counts are more common.

5. **Exploring Categorical Features**:
   - For categorical data, We have smartly chosen to remove features with high cardinality to simplify the analysis. Then, we have visualized the remaining categorical features in a grid of plots, displaying the relative frequency of each category within these features. These visualizations help to quickly grasp which categories are most or least common.

there are some important thing I want to mention here: 


# Cloudiness in Illinois

Illinois has never been considered the Sunshine State. However, cities in Illinois are considered in the middle ranks of U.S. cities in terms of cloudiness. They are neither as sunny as Las Vegas nor as cloudy as Seattle.

Here is a breakdown of the number of days with clear, partly cloudy, and cloudy skies for select cities in Illinois with long-term observations. The daily average sky cover, based on hourly weather observations of cloud types and coverage (in tenths), is classified into three classes:
- Clear: daily average sky cover ranging from 0 to 3 tenths
- Partly cloudy: daily average sky cover from 4 to 6 tenths
- Cloudy: daily average sky cover from 7 to 10 tenths

## Annual Average Number of Days by City

| City       | Clear | Partly Cloudy | Cloudy |
|------------|-------|----------------|--------|
| Cairo      | 113   | 104            | 149    |
| Chicago    | 84    | 105            | 176    |
| Moline     | 101   | 100            | 164    |
| Peoria     | 95    | 97             | 172    |
| Rockford   | 93    | 98             | 174    |
| Springfield| 104   | 94             | 167    |

You can also visit the [National Centers for Environmental Information](https://www.ncdc.noaa.gov/) for more information on cloudiness of U.S. cities.


but as you can see in our dataset the Clear weather is about 78.5% of our data which means most of the accident happened in clear way and we can see from the report that there are only 84 days with clear weather condition in chicago, so this means that people maybe care more about driving in Cloudy day so there are less accident happening with this condition.
<\span>

<br>
<br>

## 2.3. Explore the Data

2.3.1. Continuous Features Tabular Report<br>
2.3.2. Categorical Features Tabular Report <br>
2.3.3. Correlations Matrix<br>
2.3.4. HeatMap<br>
<br>


In [ ]:
# 2.3.1. Continuous Features Tabular Report

def get_continuous_features_tabular_report(df):
    # continuous_features_tabular_report_df = df.describe(include=['number'])
    return df.describe(include=['number'])

# Transpose - More friendly print
continuous_features_transpose_tabular_report_df = get_continuous_features_tabular_report(df).transpose()

continuous_features_transpose_tabular_report_df


<span style="color:cyan">

##### 2.3.1. Continuous Features Tabular Report
##### Analysis

Using tabular report, we can see that several issues on the data quality, examples:
* LANE_CNT max value is "1.191625e+06" - This not possible at all.
* LATITUDE min value is "0" - This is not possible since this is out of the range of Chicago city.

Deeper and more detailed analysis will follow in the next sections.

<br>
<br>
<br>


</span>

In [ ]:
# 2.3.2. Categorical Features Tabular Report

def get_categorical_features_tabular_report(df):
    return df.describe(exclude=['number'])

# Transpose - More friendly print
categorical_features_transpose_tabular_report_df = get_categorical_features_tabular_report(df).transpose()

categorical_features_transpose_tabular_report_df


<span style="color:cyan">

##### 2.3.2. Categorical Features Tabular Report
##### Analysis

The categorical features tabular gives important insights especially for unique and count.
We can immediately detect features with high dimensionality and features that mostly missing (need to be dropped).

Deeper and more detailed analysis will follow in the next sections.

<br>
<br>
<br>

</span>

In [ ]:
# 2.3.3. Correlation insights

def get_correlation_insights(df):
    # Get the relevant columns
    subset_df = df[get_continuous_features_tabular_report(df).columns]
    return subset_df.corr()

correlation_matrix = get_correlation_insights(df)
correlation_matrix

In [ ]:
# 2.3.4. HeatMap

def draw_heatmap(correlation_matrix):
    # Create a heatmap using seaborn
    plt.figure(figsize=(16, 14))
    return sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.8)

draw_heatmap(correlation_matrix)

<span style="color:cyan">

##### 2.3.3. Correlation insights + 2.3.4. HeatMap
##### Analysis

The categorical features tabular gives important insights especially for unique and count.
We can immediately detect features with high dimensionality and features that mostly missing (need to be dropped).

Deeper and more detailed analysis will follow in the next sections.

<br>
<br>
<br>

</span>

<br>
<br>

## 2.4. Verify Data Quality

2.4.1. Missing Values Summary<br>
2.4.2. Irregular cardinality<br>
* Features with a cardinality of 1.<br>
* Too high cardinality for categorical features.<br>
* Too low cardinality for continous features.<br>

2.4.3. Outliers<br>
2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data.<br>

<br>


In [ ]:
# 2.4.1. Missing Values Summary

# Get missing values details
# Retrun value per feature as a numeric value and a percentage
def get_missing_values_details(my_df):
    # Check for missing values in each column
    missing_values = my_df.isnull().sum()
    missing_values_sorted = missing_values.sort_values(ascending=False)
    # Check the percentage of missing values for each column
    missing_percentage_sorted = (missing_values_sorted / len(my_df)) * 100
    return missing_values_sorted, missing_percentage_sorted


def get_completeness_report(my_df):
    missing_values_sorted, missing_percentage_sorted = get_missing_values_details(my_df)
    # Create a DataFrame to summarize the completeness
    completeness_report = pd.DataFrame({
        'Missing Values': missing_values_sorted,
        'Missing Percentage': missing_percentage_sorted
    })
    return completeness_report

completeness_report = get_completeness_report(df)
completeness_report


<span style="color:cyan">

##### 2.4.1. Missing Values Summary
##### Analysis

As observed in the table above, the following values has very high missing values (more than 30%).

* WORKERS_PRESENT_I
* DOORING_I
* WORK_ZONE_TYPE
* WORK_ZONE_I
* PHOTOS_TAKEN_I
* STATEMENTS_TAKEN_I
* NOT_RIGHT_OF_WAY_I
* CRASH_DATE_EST_I
* INTERSECTION_RELATED_I
* LANE_CNT
* HIT_AND_RUN_I

<br>

Other columns with missing values we will drop instances or do imputation. Details will follow in the preprocessing section.

<br>
<br>
<br>

</span>

In [ ]:
# 2.4.2. Irregular cardinality - Features with a cardinality of 1

IRREGULAR_CARD_SINGLE = 1             # To detect features with cardinality of 1

def get_irregular_cardinality_with_single_cardinality(my_df):
    unique_counts = my_df.nunique()
    unique_counts_single_threshold = unique_counts[unique_counts == IRREGULAR_CARD_SINGLE]
    return pd.DataFrame(unique_counts_single_threshold)


get_irregular_cardinality_with_single_cardinality(df)


In [ ]:
# 2.4.2. Irregular cardinality - Too high cardinality for categorical features

IRREGULAR_CARD_MAX_THRESHOLD = 40     # To detect categorical features with high cardinality

def get_unique_counts_exceed_max_threshold_categorical(my_df):
    cardinality = my_df.select_dtypes(exclude=['number']).apply(lambda x: x.nunique())
    high_cardinality_cols = cardinality[cardinality > IRREGULAR_CARD_MAX_THRESHOLD].index.tolist()
    # print("High Cardinality Columns:", high_cardinality_cols)
    return high_cardinality_cols

high_cardinality_cols = get_unique_counts_exceed_max_threshold_categorical(df)
# Categorical Features with carinality higher than IRREGULAR_CARD_MAX_THRESHOLD
print(f"Categorical Features with carinality higher than {IRREGULAR_CARD_MAX_THRESHOLD} cardinality are: {high_cardinality_cols}")

In [ ]:
# 2.4.2. Irregular cardinality - Too low cardinality for continuous features

IRREGULAR_CARD_MIN_THRESHOLD = 10      # To detect continuous features with low cardinality

def get_unique_counts_exceed_min_threshold_continuous(my_df):
    continuous = my_df.select_dtypes(include=['number']).apply(lambda x: x.nunique())
    high_cardinality_cols = continuous[continuous < IRREGULAR_CARD_MIN_THRESHOLD].index.tolist()
    # print("High Cardinality Columns:", high_cardinality_cols)
    return high_cardinality_cols

get_unique_counts_exceed_min_threshold_continuous(df)
# df['PRIM_CONTRIBUTORY_CAUSE'].head()

<span style="color:cyan">

##### 2.4.2.  - Irregular Cardinality
##### Analysis

**Features with a cardinality of 1**<br>
INJURIES_UNKNOWN feature has cardinality of 1. It is useless in the modeling phase, and overall in the prediction model. it will be dropped in the preprocessing phase.

<br>

**Too high cardinality for categorical features**<br>

For the following features - please find details and plan about each one.

* CRASH_RECORD_ID - Will be dropped, this is just ID.
* CRASH_DATE - It makes sense DateTime format highly varies, however, we do feature selection and store data in numeric values (e.g day of the week, month, year, ...).
* DATE_POLICE_NOTIFIED - Will be dropped.
* STREET_NAME - Will be dropped. To detect location we have longtiude and latitude.
* LOCATION - This is duplicated feature. Longitude and Latitude provides same information but in numeric format. We will drop it.
* PRIM_CONTRIBUTORY_CAUSE - Has cadrinality of 40, we will do feature engineering and map to 5 values (need to agree with the team).
* SEC_CONTRIBUTORY_CAUSE - Has cadrinality of 40, we will do feature engineering and map to 5 values (need to agree with the team).
<br>
Please notice PRIM_CONTRIBUTORY_CAUSE and SEC_CONTRIBUTORY_CAUSE have exact same classes.

<br>

**Too low cardinality for continuous features**<br>

For the following features - please find details and plan about each one.
* INJURIES_FATAL - Makes sense it is low number (which is good! Less people died)
* INJURIES_UNKNOWN - Same here, this is indication for unkown injuries.
* CRASH_DAY_OF_WEEK - Number vary between 1-7 (Mon, Tue, ... Sun). So value makes sense.

Nothing to do in this section.

<br>
<br>
<br>

</span>

In [ ]:
# 2.4.3. Outliers

def get_outliers_list_of_indices(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    ls = df.index[ (df[ft] < lower_bound) | (df[ft] > upper_bound) ]
    
    return ls

# To store the indicies
outliers_index_list = []
# To store the features (set, so no duplications)
outliers_features_set = set()
numeric_columns = df.select_dtypes(include=['number']).columns

for feature in numeric_columns:
    indicies_list = get_outliers_list_of_indices(df, feature)
    outliers_index_list.extend(indicies_list)
    if len(indicies_list) != 0:
        outliers_features_set.add(feature)

print(f"There are {len(outliers_index_list)} outliers")
print(f"Features set are {outliers_features_set}")

<span style="color:cyan">

##### 2.4.3.  - Outliers
##### Analysis

Outliers inital result shows the following features are outliers:
* STREET_NO
* POSTED_SPEED_LIMIT
* INJURIES_REPORTED_NOT_EVIDENT
* LONGITUDE
* LANE_CNT
* BEAT_OF_OCCURRENCE
* INJURIES_TOTAL
* INJURIES_INCAPACITATING
* INJURIES_NON_INCAPACITATING
* INJURIES_NO_INDICATION
* LATITUDE
* INJURIES_FATAL
* NUM_UNITS

Let's understand if they are really outliers, or the values are valid (e.g. street number is valid to be in a very big range).

<br>
<br>
<br>

</span>

In [ ]:
get_continuous_feature_details(df, 'STREET_NO')
get_continuous_feature_details(df, 'POSTED_SPEED_LIMIT')
get_continuous_feature_details(df, 'INJURIES_REPORTED_NOT_EVIDENT')
get_continuous_feature_details(df, 'LONGITUDE')
get_continuous_feature_details(df, 'LANE_CNT')
get_continuous_feature_details(df, 'BEAT_OF_OCCURRENCE')
get_continuous_feature_details(df, 'INJURIES_TOTAL')
get_continuous_feature_details(df, 'INJURIES_INCAPACITATING')
get_continuous_feature_details(df, 'INJURIES_NON_INCAPACITATING')
get_continuous_feature_details(df, 'INJURIES_NO_INDICATION')
get_continuous_feature_details(df, 'LATITUDE')
get_continuous_feature_details(df, 'INJURIES_FATAL')
get_continuous_feature_details(df, 'NUM_UNITS')
pass # To avoid printing of the previous line and keep only formatted-pretty prints

<span style="color:cyan">

##### 2.4.3.  - Outliers (Continued)
##### Analysis

| Feature                        | Unique Values Len | Unique Values Example                  | Max Value    | Min Value    | Comments                                                                 | Explanation                                        |
|--------------------------------|-------------------|----------------------------------------|--------------|--------------|--------------------------------------------------------------------------|----------------------------------------------------|
| STREET_NO                      | 11723             | 0, 1, 2, 3, 4, ..., 13787, 13795, 13799, 34453, 451100 | 451100       | 0            | Street numbers in Chicago, large range due to city size and data errors. | Street values can highly vary, it doesn't follow math logic      |
| POSTED_SPEED_LIMIT             | 46                | 0, 1, 2, 3, 4, ..., 62, 63, 65, 70, 99 | 99           | 0            | Posted speed limits, including possibly erroneous values.                | Speed limits can highly vary. We checked all values count to verify no outliers in respect to allowed speed - e.g. no instance with value 500 (please refer to the function dump_feature_frequency_to_a_file)       |
| INJURIES_REPORTED_NOT_EVIDENT  | 14                | 0.0, 1.0, 2.0, 3.0, 4.0, ..., 9.0, 10.0, 11.0, 15.0 | 15.0         | 0.0          | Number of non-evident injuries reported per crash.                       | Range is reasonable for severe accidents.           |
| LONGITUDE                      | 299263            | -87.936192947, ..., -87.524587387, 0.0 | 0.0          | -87.936192947| Longitude values in Chicago, including erroneous 0.0 entries.            | 0.0 likely represents missing or incorrectly entered data. It will be handled in data preprocessing phase. |
| LANE_CNT                       | 42                | 0.0, 1.0, 2.0, 3.0, 4.0, ..., 1191625.0| 1191625.0    | 0.0          | Number of lanes, with some unrealistic values due to errors.             | Extreme max value due to data entry errors.        |
| BEAT_OF_OCCURRENCE             | 277               | 111.0, ..., 2535.0, 6100.0             | 6100.0       | 111.0        | Police beat codes, including possibly erroneous 6100.0.                 | This is police internal codes and can't be treated as outliers          |
| INJURIES_TOTAL                 | 21                | 0.0, 1.0, ..., 19.0, 21.0              | 21.0         | 0.0          | Total number of injuries per crash.                                      | Range is within expected limits for traffic accidents. |
| INJURIES_INCAPACITATING        | 11                | 0.0, 1.0, ..., 8.0, 10.0               | 10.0         | 0.0          | Number of incapacitating injuries per crash.                             | Within expected limits, considering severe cases.   |
| INJURIES_NON_INCAPACITATING    | 20                | 0.0, 1.0, ..., 19.0, 21.0              | 21.0         | 0.0          | Number of non-incapacitating injuries per crash.                         | Range reflects possible severe accidents.           |
| INJURIES_NO_INDICATION         | 49                | 0.0, 1.0, ..., 50.0, 61.0              | 61.0         | 0.0          | People involved in crash with no injuries reported.                      | High values for large accidents, not outliers.     |
| LATITUDE                       | 299300            | 0.0, 41.644670132, ..., 42.022779861   | 42.022779861 | 0.0          | Latitude values in Chicago, including erroneous 0.0 entries.             | 0.0 likely represents missing or incorrectly entered data. It will be handled in data preprocessing phase. |
| INJURIES_FATAL                 | 6                 | 0.0, 1.0, ..., 3.0, 4.0                | 4.0          | 0.0          | Number of fatal injuries per crash.                                      | Fatalities are rare, but values are within expected limits. |
| NUM_UNITS                      | 17                | 1, 2, ..., 16, 18                      | 18           | 1            | Number of units involved in a crash (e.g. cars, motocycles, ...) | Not an outlier, the highest values sounds reasonable |




From the table above  we can notice that all features don't have outliers, except the following features:
* LANE_CNT
* LONGITUDE
* LATITUDE


Doing research in Google Maps and Google for Chicago city LATITUDE and LONGITUDE range (also NaN, but will be addressing in missing values section), here is our findings:<br>
LATITUDE valid range: [41.640, 42.023]<br>
LONGITUDE valid range: [-87.940, -87.524]<br>

We will address that again in the Data Preprocessing section to clean up the out of range values.

<br>
<br>
<br>

</span>


In [ ]:
# 2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data

def consistency_check_duplicated_instances(df):
    duplicate_rows = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicate_rows}")
    # If you want to actually see the duplicate rows, you can use:
    if duplicate_rows > 0:
        print(df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist()))

consistency_check_duplicated_instances(df)


<span style="color:cyan">

##### 2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data
##### Analysis

No issues found.

<br>
<br>
<br>

</span>

<br>
<br>
<br>

# 3. CRISP-DM: Data Preparation

3.1. Missing Values<br>
3.2. Duplicated Columns - LOCATION is the pair LATITUDE and LONGITUDE<br>
3.3. Drop CRASH_ID - ID column<br>
3.4. Irregular Cardinality<br>
3.5. Outliers<br>
3.6. Feature Egnineering - Extract Date Infromation<br>

More details in each section.

In [ ]:
# Get a copy from the original dataset to start preprocessing phase

df_cleaned = df.copy()


In [ ]:
# 3.1. Missing Values (Part-1) - Drop

# Set the threshold for dropping columns
MISSING_THRESHOLD = 30.0

# Identify columns that have missing value percentage greater than the threshold
columns_to_drop = completeness_report[completeness_report['Missing Percentage'] >= MISSING_THRESHOLD].index

# Drop these columns from the DataFrame
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

# Now, df has the columns dropped where the missing value percentage was higher than 30%
print("Features with missing instances higher than 30% that has been dropped")
columns_to_drop

In [ ]:
# 3.1. Missing Values (Part-2) - Special handle for LATITUDE and LONGITUDE

def drop_missing_instances(my_df, my_feature):
    df_cleaned = my_df.dropna(subset=[my_feature])
    return df_cleaned
df_cleaned = drop_missing_instances(df_cleaned, 'LATITUDE')
df_cleaned = drop_missing_instances(df_cleaned, 'LONGITUDE')
pass


In [ ]:
# 3.1. Missing Values (Part-3) - Imputation for values

# Identify columns that have missing value percentage less than the imputation threshold
columns_to_impute = completeness_report[completeness_report['Missing Percentage'] < MISSING_THRESHOLD].index

# Loop through the columns and perform imputation
for column in columns_to_impute:
    if df_cleaned[column].dtype == 'numeric':
        # Impute numerical columns with the mean value
        df_cleaned[column].fillna(df_cleaned[column].mean(), inplace=True)
    else:
        # Impute categorical columns with the mode value (the most frequent value)
        df_cleaned[column].fillna(df_cleaned[column].mode()[0], inplace=True)


In [ ]:
# 3.1. Missing Values (Part-43) - Print for visibility of current situation

completeness_report = get_completeness_report(df_cleaned)
completeness_report

<span style="color:cyan">

##### 3.1. Missing Values
##### Analysis


**Missing**

For missing values higher than 30%, we drop them.

* WORKERS_PRESENT_I
* DOORING_I
* WORK_ZONE_TYPE
* WORK_ZONE_I
* PHOTOS_TAKEN_I
* STATEMENTS_TAKEN_I
* NOT_RIGHT_OF_WAY_I
* CRASH_DATE_EST_I
* INTERSECTION_RELATED_I
* LANE_CNT
* HIT_AND_RUN_I

Please refer to missing values report in the Data Understanding section for more details.

<br>

**Special handle for LATITUDE and LONGITUDE**
Missing values for LATITUDE and LONGITUDE is less than 1%, we will drop these instances.
Imputation is bad for this case, we prefer to have high accuracy and valid data for the location related params.

<br>

**Imputation for values**

For missing values lower than 30% - we do imputation. To be more accurate, in the terms of our dataset, missing values under 30% ranges between 0.1%-3% missing values so imputation is a very reasonable choice in this case.

<br>
<br>
<br>

</span>

In [ ]:
# 3.2. Duplicated Columns - LOCATION is the pair LATITUDE and LONGITUDE

df_cleaned = df_cleaned.drop('LOCATION', axis=1)



<span style="color:cyan">

##### 3.2. Duplicated Columns - LOCATION is the pair LATITUDE and LONGITUDE
##### Analysis

LOCATION is the pair of values of LONGITUDE and LATITUDE, we choose to drop it due to two reasons:
1. It is duplication of other features (aggregation of LONGITUDE and LATITUDE in pairs will result in LOCATION).
2. It is categorical feature, with high class dimensionality - meaning, in the encoding phase to prepare for modeling, it will generate very big number of derived features. In the other hand, LONGITUDE and LATITUDE is numeric and much easier to the handling in modeling phase.

<br>
<br>
<br>

</span>

In [ ]:
# 3.3. Drop CRASH_ID - ID column

df_cleaned = df_cleaned.drop('CRASH_RECORD_ID', axis=1)


<span style="color:cyan">

##### 3.3. Drop CRASH_ID - ID column
##### Analysis

It is just ID of the crash, and has no contributation to the prediction model.
<br>
This is also could be dropped as part of irregular cardinality.
<br>
<br>
<br>

</span>

In [ ]:
# 3.4. Irregular Cardinality

# Drop feature with cardinality of 1
df_cleaned = df_cleaned.drop('INJURIES_UNKNOWN', axis=1)

# Too high cardinality for categorical features
# CRASH_RECORD_ID - Already dropped.
# df_cleaned = df_cleaned.drop('CRASH_DATE', axis=1) # Will be deleted later after feature engineering
df_cleaned = df_cleaned.drop('DATE_POLICE_NOTIFIED', axis=1)
df_cleaned = df_cleaned.drop('STREET_NAME', axis=1)




<span style="color:cyan">

##### 3.4. Irregular Cardinality
##### Analysis

<br>

**Drop feature with cardinality of 1**


INJURIES_UNKNOWN feature has cardinality of 1 --> Drop.

<br>

**Too high cardinality for categorical features**<br>

For the following features - please find details and plan about each one.

In addition to what was dropped in 3.2. (LOCATION), 3.3. (MONTH), and 3.4 (CRASH_ID) - The following will be dropped.
* CRASH_DATE - Will be handled later after feature selection
* DATE_POLICE_NOTIFIED
* STREET_NAME - Will be dropped. To detect location we have longtiude and latitude.
* LOCATION - This is duplicated feature. Longitude and Latitude provides same information but in numeric format. We will drop it.

<br>

**Too low cardinality for continuous features**<br>

Nothing to do in this section.

<br>
<br>
<br>

</span>

In [ ]:
# Section 3.5. - Outliers

print("========================== Values Before ==========================")
get_continuous_feature_details(df_cleaned, 'LATITUDE')
get_continuous_feature_details(df_cleaned, 'LONGITUDE')
# get_continuous_feature_details(df_fix_range_issues, 'LANE_CNT') # Already dropped due to high missing values

# To remove values for my_feature outside a given range range (including min_value and max_value)
def remove_invalid_values_outside_given_range(my_df, my_feature, min_value, max_value):
    # Create a mask for values within the range
    valid_mask = (my_df[my_feature] >= min_value) & (my_df[my_feature] <= max_value)
    # Apply the mask to the DataFrame
    filtered_df = my_df[valid_mask]
    return filtered_df

# Define the valid range for LATITUDE
min_latitude = 41.640
max_latitude = 42.023
df_cleaned = remove_invalid_values_outside_given_range(df_cleaned, 'LATITUDE', min_latitude, max_latitude)

# Define the valid range for LONGITUDE
min_longitude = -87.940
max_longitude = -87.524
df_cleaned = remove_invalid_values_outside_given_range(df_cleaned, 'LONGITUDE', min_longitude, max_longitude)


# Already dropped due to high missing values count
# min_lane_count = 0
# max_lane_count = 10
# df_fix_range_issues = remove_invalid_values_outside_given_range(df_fix_range_issues, 'LANE_CNT', min_lane_count, max_lane_count)


print("========================== Values After ==========================")
get_continuous_feature_details(df_cleaned, 'LATITUDE')
get_continuous_feature_details(df_cleaned, 'LONGITUDE')
# get_continuous_feature_details(df_fix_range_issues, 'LANE_CNT')

get_continuous_features_tabular_report(df_cleaned).transpose()


<span style="color:cyan">

##### 3.5. Outliers
##### Analysis

This section addresses problems mostly in ranges for special features.
For example, week can't be more than 7 days.
Let's go over the descriptive features and provide insights.

We already described that above, but we address it here again.

| Feature            | Unique Values Len | Unique Values Example    | Max Value | Min Value | Comments                |
|--------------------|-------------------|--------------------------|-----------|-----------|-------------------------|
| CRASH_HOUR         | 24 |  [0 1 2 ... 23]  |  23 | 0 | Time of the crash (0-23)|
| CRASH_DAY_OF_WEEK  | 7 | [1 2 3 4 5 6 7] | 7 | 1 | Day of the week (1-7)   |
| CRASH_MONTH        | 12 | [1 2 3 ... 12] |  12  |  1   | Month of the crash (1-12)|
| LATITUDE           |  299299 |  [ 0 41.64467013 41.64469152 ... 42.02273632 ]      |    42.022779861       |     0.0      | Geographical latitude   |
| LONGITUDE          |  299262  | [-87.93619295 -87.93587692 ... 0] |   0.0        |     -87.936192947      | Geographical longitude  |

<br>

As the table illustrates, the following features have valid values:<br>
* CRASH_HOUR
* CRASH_DAY_OF_WEEK
* CRASH_MONTH

<br>

But the following features has values out of range / outliers in respect to each one type:
* LATITUDE - Min value is 0.0
* LONGITUDE - Max value is 0.0

<br>

Doing research in Google Maps and Google for Chicago city LATITUDE and LONGITUDE range (also NaN, but will be addressing in missing values section), here is our findings:<br>
LATITUDE valid range: [41.640, 42.023]<br>
LONGITUDE valid range: [-87.940, -87.524]<br>

So for LATITUDE and LONGITUDE we make sure we are in the valid range.

<br>
<br>
<br>

</span>

In [ ]:
# 3.6. Feature Egnineering - Extract Date Infromation

# Convert CRASH_DATE to datetime
df_cleaned['CRASH_DATE'] = pd.to_datetime(df_cleaned['CRASH_DATE'])

# Extract components from CRASH_DATE
df_cleaned['YEAR'] = df_cleaned['CRASH_DATE'].dt.year
df_cleaned['DAY'] = df_cleaned['CRASH_DATE'].dt.day

# # Drop the CRASH_DATE
df_cleaned = df_cleaned.drop('CRASH_DATE', axis=1)


<span style="color:cyan">

##### 3.6. Feature Egnineering - Extract Date Infromation
##### Analysis

Extraction of YEAR and DAY from DateTime format of CRASH_DATE.<br>
Please notice for month, we already have CRASH_MONTH.<br>

<br>
<br>
<br>

</span>

In [ ]:
# Debug Only

# dump_feature_frequency_to_a_file(df_cleaned ,'POSTED_SPEED_LIMIT')
# dump_feature_frequency_to_a_file(df_cleaned ,'DAMAGE')
# dump_feature_frequency_to_a_file(df_cleaned ,'CRASH_TYPE')
# dump_feature_frequency_to_a_file(df_cleaned ,'REPORT_TYPE')


In [ ]:
# 3.7. Feature Engineering - Contributing Causes

# Commented - Need to discuss.

# df_fe_contributing_cause = df_outliers.copy()


# # Define the mapping of original causes to the 5 broad categories
# category_mapping = {
#     'Traffic Sign and Signal Violations': [
#         'DISREGARDING STOP SIGN', 'DISREGARDING TRAFFIC SIGNALS', 'DISREGARDING YIELD SIGN',
#         'DISREGARDING OTHER TRAFFIC SIGNS', 'DISREGARDING ROAD MARKINGS',
#         'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT',
#         'TURNING RIGHT ON RED'
#     ],
#     'Driver Behavior and Condition': [
#         'DISTRACTION - FROM INSIDE VEHICLE', 'DISTRACTION - FROM OUTSIDE VEHICLE',
#         'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',
#         'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
#         'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
#         'PHYSICAL CONDITION OF DRIVER', 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',
#         'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)', 'TEXTING',
#         'CELL PHONE USE OTHER THAN TEXTING'
#     ],
#     'Vehicle Operation and Conditions': [
#         'FAILING TO REDUCE SPEED TO AVOID CRASH', 'FAILING TO YIELD RIGHT-OF-WAY',
#         'FOLLOWING TOO CLOSELY', 'IMPROPER BACKING', 'IMPROPER LANE USAGE',
#         'IMPROPER OVERTAKING/PASSING', 'IMPROPER TURNING/NO SIGNAL', 'DRIVING ON WRONG SIDE/WRONG WAY',
#         'EQUIPMENT - VEHICLE CONDITION'
#     ],
#     'Environmental and External Conditions': [
#         'ANIMAL', 'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST', 'OBSTRUCTED CROSSWALKS',
#         'ROAD CONSTRUCTION/MAINTENANCE', 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS',
#         'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 'WEATHER', 'RELATED TO BUS STOP'
#     ],
#     'Speed and Compliance': [
#         'EXCEEDING AUTHORIZED SPEED LIMIT', 'EXCEEDING SAFE SPEED FOR CONDITIONS',
#         'PASSING STOPPED SCHOOL BUS'
#     ]
# }

# # Reverse the mapping to make it easier to map each cause to its new category
# reversed_mapping = {}
# for category, causes in category_mapping.items():
#     for cause in causes:
#         reversed_mapping[cause] = category

# # Default category for those not listed
# default_category = 'Miscellaneous'

# # Create a new columns with the mapped categories
# df_fe_contributing_cause['PRIM_CONTRIBUING_CAUSE_UPDATED'] = df_fe_contributing_cause['PRIM_CONTRIBUTORY_CAUSE'].map(lambda x: reversed_mapping.get(x, default_category))
# df_fe_contributing_cause['SEC_CONTRIBUING_CAUSE_UPDATED'] = df_fe_contributing_cause['SEC_CONTRIBUTORY_CAUSE'].map(lambda x: reversed_mapping.get(x, default_category))

# # Drop the old columns
# df_fe_contributing_cause = df_fe_contributing_cause.drop('PRIM_CONTRIBUTORY_CAUSE', axis=1)
# df_fe_contributing_cause = df_fe_contributing_cause.drop('SEC_CONTRIBUTORY_CAUSE', axis=1)

# df_fe_prim_contributing_cause['PRIM_CONTRIBUING_CAUSE_UPDATED'].head(5)
# df_fe_prim_contributing_cause['SEC_CONTRIBUING_CAUSE_UPDATED'].tail(5)


<span style="color:cyan">

##### 3.7. Feature Engineering - Contributing Causes
##### Analysis

PRIM_CONTRIBUTORY_CAUSE and SEC_CONTRIBUTORY_CAUSE each one has excatly 40 possible classes which are the same.
We map the to 5 catagories as illustrated in the code.

Purpose of this is two reasons:
1. To generalize the reason of contributing factors, and see their impact on traffic injury prediction.
2. To reduce high dimensionality.

<br>
<br>
<br>

</span>


In [ ]:

print(f"Shape Before Preprocessing: {df.shape}")
print(f"Shape After  Preprocessing: {df_cleaned.shape}")


<br>
<br>
<br>
<br>
<br>



# 4. CRISP-DM: Modeling

**Section Overview**

4.1. Import relevant ML libs for Modeling.<br>
4.2. Formaluize a Numerical Measure for the Target Variable(s).<br>
4.3. Undersampling - To fix over-representation in the dataset (unbalanced data).<br>
4.4. Modeling with Random Forest.<br>
4.5. Modeling with Logistic  Regression.<br>
4.6. Modeling with DNN.<br>
4.7. Modeling with KNN.<br>

In [ ]:
# 4.1. Import relevant ML libs for Modeling
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

In [ ]:
# 4.2. Formaluize a Numerical Measure for the Target Variable(s)

df_to_model = df_cleaned.copy()

# Building our numerical metrics for measuring the Severity of injuries based on the reported injuries in the dataset
df_to_model['SEVERITY_OF_INJURIES'] = ((0.3 * df_to_model['INJURIES_NON_INCAPACITATING'] + 
                                 0.6 * df_to_model['INJURIES_INCAPACITATING'] + 
                                 # Assuming we might use 'INJURIES_INCAPACITATING' or another column for fatal injuries representation
                                 0.1 * df_to_model['INJURIES_NO_INDICATION'] + df_to_model['INJURIES_FATAL']) / 
                                ((df_to_model['INJURIES_NON_INCAPACITATING'] + 
                                      df_to_model['INJURIES_INCAPACITATING'] + 
                                      # Again, assuming a placeholder for fatal injuries if needed
                                      df_to_model['INJURIES_NO_INDICATION']+df_to_model['INJURIES_FATAL'])))

# Code to create the "INJURY_CLASS" feature based on "SEVERITY_OF_INJURIES"
df_to_model['INJURY_ClASS'] = df_to_model['SEVERITY_OF_INJURIES'].apply(lambda x: 'HIGH INJURY' if x > 0.2 else 'LIGHT INJURY')

# Code to create the "SEVERITY_CLASS" feature based on "INJURIES_FATAL" and "INJURIES_INCAPACITATING"
df_to_model['SEVERITY_CLASS'] = df_to_model.apply(lambda x: 'HIGH SEVERITY' if x['INJURIES_FATAL'] > 0 or x['INJURIES_INCAPACITATING'] > 0 else 'LOW SEVERITY', axis=1)

# Display the first few rows to see the new feature
df_to_model[['INJURIES_NON_INCAPACITATING','INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NO_INDICATION', 'SEVERITY_OF_INJURIES','INJURY_ClASS','SEVERITY_CLASS']].head()

<span style="color:cyan">

##### 4.2. Formaluize a Numerical Measure for the Target Variable(s)
##### Analysis

Our target variable is represented via 4 variables were each one giving the following indications:
* INJURIES_NO_INDICATION - No injury or light reported.
* INJURIES_NON_INCAPACITATING - Medium injury.
* INJURIES_INCAPACITATING - Heavy.
* INJURIES_FATAL - Death.

To build a mesurement to estimate the fatality of the crash injury crash, we define a weighted equation for each class and normalize it.

<br>
<br>
<br>

</span>

In [ ]:
# Debug only - print columns
df_to_model.columns

In [ ]:

# Preparing features and target variables 
# X = df_to_model.drop(['SEVERITY_OF_INJURIES','INJURIES_TOTAL' ,'INJURIES_INCAPACITATING', 'MOST_SEVERE_INJURY','INJURY_ClASS','SEVERITY_CLASS',
#                   'INJURIES_NON_INCAPACITATING', 'SEVERITY_CLASS','INJURIES_NO_INDICATION','CRASH_DATE',
#                   'INJURIES_REPORTED_NOT_EVIDENT','INJURIES_FATAL','STREET_NAME'],axis = 1)


X = df_to_model.drop(['SEVERITY_OF_INJURIES', 'INJURIES_TOTAL', 'INJURIES_INCAPACITATING', 'MOST_SEVERE_INJURY', 
                      'INJURY_ClASS', 'SEVERITY_CLASS', 'INJURIES_NON_INCAPACITATING', 'INJURIES_NO_INDICATION',
                      'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_FATAL'], axis=1)
y = df_to_model['SEVERITY_CLASS']
X = pd.get_dummies(X)

# Initialize the random under-sampler due to unbalanced dataset
rus = RandomUnderSampler(random_state=42)

# Resample the dataset
X_resampled, y_resampled = rus.fit_resample(X, y)

<span style="color:cyan">

1. **Drops Irrelevant Columns**: Removes columns related to the severity and specifics of injuries, among others, that are not used as features for the prediction model. This step focuses the model on relevant predictors and avoids data leakage from features too closely related to the target variable.

2. **Target Variable Selection**: Sets the `SEVERITY_CLASS` column as the target variable (`y`), which likely represents the classification outcome we want to predict.

3. **One-Hot Encoding**: Applies `pd.get_dummies()` to convert categorical variables into a format that can be provided to machine learning algorithms. This step is crucial for models that require numerical input.

4. **Addressing Class Imbalance**: Initializes and applies a Random Under-Sampler (`RandomUnderSampler`) to the dataset. This technique mitigates the issue of a highly imbalanced dataset by reducing the size of the over-represented class, leading to a more balanced dataset that can improve model performance. Choosing under-sampling over over-sampling is a strategic decision to prevent potential data leakage that could occur by duplicating entries of the under-represented class, which could give the model an unfair advantage by "seeing" duplicated data during training.

</span>

In [ ]:


def split_data(X, y, test_size=0.2, random_state=42):
    
    # First, split into train+validation and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Then split train+validation into train and validation
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=random_state)
    
    return X_train, X_val, X_test, y_train, y_val, y_test


# logestic_regression
def implement_logistic_regression(X_train, X_val, X_test, y_train, y_val, y_test):

    # hyperparameter tuning for logestic regression model
    best_accuracy = 0
    best_c = None
    for C in np.logspace(-4, 4, 20):
        log_reg = LogisticRegression(C=C, max_iter=10000, random_state=42)
        log_reg.fit(X_train, y_train)
        accuracy = log_reg.score(X_val, y_val)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_c = C

    # Initialize the Logistic Regression model with balanced class weights
    log_reg = LogisticRegression(C=best_c, max_iter=10000, random_state=42)

    # Fitting the model to the training data
    log_reg.fit(X_train, y_train)

    # Predicting on the test set
    y_pred = log_reg.predict(X_test)

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    classification_report_results = classification_report(y_test, y_pred)

    print(f"Best C: {best_c}")
    print("Accuracy of linear regressor:", accuracy)
    print("\nClassification Report:\n", classification_report_results)
        
    return log_reg


# Random Forest
def implement_random_forest(X_train, X_val, X_test, y_train, y_val, y_test):


    # Hypterparameter tuning for random forest model
    best_accuracy = 0
    best_n_estimators = None
    for n_estimators in range(10, 100, 10):
        rf_classifier = RandomForestClassifier(n_estimators=n_estimators, class_weight='balanced', random_state=42)
        rf_classifier.fit(X_train, y_train)
        accuracy = rf_classifier.score(X_val, y_val)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_n_estimators = n_estimators


    rf_classifier = RandomForestClassifier(n_estimators=best_n_estimators, random_state=42)
    rf_classifier.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = rf_classifier.predict(X_test)

    # Predicting on the test set
    accuracy = accuracy_score(y_test, y_pred)

    # Evaluating the model
    classification_report_results = classification_report(y_test, y_pred)
    
    print(f"Best n_estimators: {best_n_estimators}")
    print("Accuracy of Random Forest Classifier:", accuracy)
    print("\nClassification Report:\n", classification_report_results)
    
    return rf_classifier

# Deep Neural Network 
def implement_DNN(X_train, X_test, y_train, y_test):
    

    # It's important to scale your input features for neural networks
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define the parameter grid to search
    param_grid = {
        'hidden_layer_sizes': [(10,10), (10,20), (20, 20)],
        'activation': ['tanh', 'relu'],
    }
    
    # Initialize GridSearchCV - note that we're using the training data for fitting
    grid_search = GridSearchCV(MLPClassifier(max_iter=100, random_state=42), param_grid, cv=5, scoring='accuracy')
    
    # Perform the grid search on the training dataset
    grid_search.fit(X_train_scaled, y_train)

    # The best_estimator_ is now the trained classifier with the best parameters found
    best_classifier = grid_search.best_estimator_

    # Predict on the test set with the best model
    y_pred = best_classifier.predict(X_test_scaled)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    classification_report_results = classification_report(y_test, y_pred)

    print("Best parameters found:", grid_search.best_params_)
    print("Accuracy of DNN model:", accuracy)
    print("\nClassification Report:\n", classification_report_results)


# K Nearest Neighbor
def implement_KNN(X_train, X_val, X_test, y_train, y_val, y_test):

    # Initialize the KNN classifier
    # It's important to scale your input features for KNN
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_val_scaled = scaler.transform(X_val)

    # Hyperparameter tuning for KNN
    best_accuracy = 0
    best_n_neighbors = None
    for n_neighbors in range(1, 21):
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train_scaled, y_train)
        accuracy = knn.score(X_val_scaled, y_val)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_n_neighbors = n_neighbors

    # n_neighbors is set to 10 as an example, but you should tune this parameter
    knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)

    # Train the model
    knn.fit(X_train_scaled, y_train)

    # Predict on the test set
    y_pred = knn.predict(X_test_scaled)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    classification_report_results = classification_report(y_test, y_pred)
    
    print(f"Best n_estimators: {best_n_neighbors}")
    print("Accuracy of KNN model:", accuracy)
    print("\nClassification Report:\n", classification_report_results)



<span style="color:cyan">

### Data Splitting Function
The `split_data` function is designed for preparing the dataset for model training and evaluation. It performs the following tasks:

1. **Data Splitting**: Initially, it splits the provided dataset into two parts: a combined training and validation set, and a separate test set. The test set size is determined by the `test_size` parameter, allowing for a portion of the data to be reserved for final evaluation.
2. **Further Splitting**: Subsequently, it further splits the combined training and validation set into distinct training and validation sets. This allows for the evaluation of model performance and hyperparameter tuning before the final test set evaluation.

### Logistic Regression Model Implementation
The `implement_logistic_regression` function handles the logistic regression model. It includes:

1. **Hyperparameter Tuning**: It iterates over different values of the regularization strength (`C`) to find the best setting based on validation set accuracy.
2. **Model Training and Evaluation**: After identifying the best hyperparameter value, it retrains the model using the entire training set and evaluates its performance on the unseen test set.

### Random Forest Model Implementation
The `implement_random_forest` function focuses on the Random Forest model. Similar to logistic regression, it involves:

1. **Hyperparameter Tuning**: This process finds the optimal number of trees (`n_estimators`) in the forest by evaluating the model's performance on the validation set.
2. **Final Model Training and Evaluation**: With the best `n_estimators` value, the model is retrained and then evaluated on the test set to report its accuracy and classification performance.

### Deep Neural Network (DNN) Implementation
The `implement_DNN` function encapsulates the process of implementing, tuning, and evaluating a Deep Neural Network model:

1. **Feature Scaling**: It scales the features using standardization, which is crucial for neural network performance.
2. **Grid Search for Hyperparameter Tuning**: Utilizes `GridSearchCV` to automate the search for the best network architecture (`hidden_layer_sizes`) and activation function over the training data.
3. **Model Evaluation**: After identifying the best hyperparameters, it evaluates the model on the test set and reports performance metrics.

### K-Nearest Neighbors (KNN) Implementation
The `implement_KNN` function addresses the KNN model implementation with:

1. **Feature Scaling**: Similar to DNN, it scales the input features to improve model performance.
2. **Hyperparameter Tuning**: Searches for the best `n_neighbors` value by evaluating different settings on the validation set.
3. **Final Model Training and Evaluation**: With the optimal `n_neighbors` value identified, it trains the KNN model and assesses its accuracy and classification performance on the test set.

</span>

In [ ]:
# Split the data
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_resampled, y_resampled)

# imbalanced data to check the accuracy of model using 0.05 percentage of data
df_to_model_frac = df_to_model.sample(frac=0.05, random_state=42)
X2 = df_to_model_frac.drop(['SEVERITY_OF_INJURIES', 'INJURIES_TOTAL', 'INJURIES_INCAPACITATING', 'MOST_SEVERE_INJURY', 
                      'INJURY_ClASS', 'SEVERITY_CLASS', 'INJURIES_NON_INCAPACITATING', 'INJURIES_NO_INDICATION',
                      'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_FATAL'], axis=1)
y2 = df_to_model_frac['SEVERITY_CLASS']
X2 = pd.get_dummies(X2)

X_train_frac, X_val_frac, X_test_frac, y_train_frac, y_val_frac, y_test_frac = split_data(X2, y2)

# Logistic Regression for imabalanced class
rf_imbalanced_classifier = implement_random_forest(X_train_frac, X_val_frac, X_test_frac, y_train_frac, y_val_frac, y_test_frac)

# Random Forest
rf_classifier = implement_random_forest(X_train, X_val, X_test, y_train, y_val, y_test)

<span style="color:cyan">



This code demonstrates the impact of handling imbalanced datasets in machine learning model performance, specifically using Random Forest classifiers:

1. **Data Splitting**: Initially, the dataset is split into training, validation, and test sets twice—once for a balanced dataset (`X_resampled`, `y_resampled`) created through undersampling and once for a fraction (5%) of the original, imbalanced dataset (`df_to_model_frac`).

2. **Model Implementation on Imbalanced Data**: A Random Forest model (`rf_imbalanced_classifier`) is trained and evaluated on the small, imbalanced fraction of the dataset. Despite achieving a high accuracy of 98.36%, the model fails to correctly predict any `HIGH SEVERITY` incidents, highlighting the issue with imbalanced data where the model is biased towards the majority class (`LOW SEVERITY`).

3. **Model Implementation on Balanced Data**: Another Random Forest model (`rf_classifier`) is then trained on the balanced dataset. This model shows a more balanced performance with an accuracy of 86.72% and is capable of identifying `HIGH SEVERITY` cases with high precision and recall, demonstrating the effectiveness of balancing the dataset through undersampling.

The stark difference in performance between the two models underscores the necessity of addressing class imbalance in datasets. While the imbalanced model might appear highly accurate at first glance, its inability to detect `HIGH SEVERITY` incidents renders it practically ineffective for scenarios where identifying such cases is crucial. Conversely, the model trained on balanced data shows significantly improved and more reliable performance across both classes, illustrating the importance of using methods like undersampling to enhance model fairness and effectiveness.

</span>


### Important Features for Random Forest Classifier
based on RF classifier I check the important features for detecting the severity of injuries in the accident.

In [ ]:
def rf_important_feature(rf_classifier):
    importances = rf_classifier.feature_importances_
    feature_names = X.columns
    feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    return feature_importances.head(20)
rf_important_feature(rf_classifier)

In [ ]:
# Logistic Regression
log_reg = implement_logistic_regression(X_train, X_val, X_test, y_train, y_val, y_test)

<span style="color:cyan">

1. **Implementation of Logistic Regression**: The `implement_logistic_regression` function is called with the training, validation, and test datasets. This function internally:
   - Performs hyperparameter tuning to find the optimal regularization strength (`C`) that leads to the best accuracy on the validation set. This is crucial for preventing overfitting while maintaining the model's ability to generalize well to new data.
   - Retrains the logistic regression model using the optimal `C` value found through tuning on the entire training dataset (including the validation data) to fully leverage all available data for training.
   - Evaluates the final model's performance on the test dataset to assess its generalization capability.

2. **Model Performance Evaluation**: The output provides detailed performance metrics for the logistic regression model:
   - The `Best C` indicates the optimal regularization strength discovered during hyperparameter tuning, in this case, `C=2`.
   - The `Accuracy of linear regressor` shows the overall percentage of correct predictions made by the model on the test set, which is approximately 86.43%. This indicates a high level of accuracy in the model's ability to classify the severity of crashes.
   - The `Classification Report` provides a breakdown of precision, recall, and F1-score for each class (`HIGH SEVERITY` and `LOW SEVERITY`). Notably, the model has high recall for `HIGH SEVERITY` cases, indicating it is particularly effective at identifying most of the high severity incidents. However, the precision for `LOW SEVERITY` is higher, suggesting the model is more reliable when it predicts an incident is of low severity.

</span>

### Important features

In [ ]:
def log_reg_important_features(log_reg):
    # Get the feature names
    feature_names = X_resampled.columns

    # Get the coefficients from the logistic regression model
    coefficients = log_reg.coef_[0]  # Assuming binary classification, hence [0]

    # Create a series to map feature names to their coefficients
    feature_importance = pd.Series(coefficients, index=feature_names)

    # Sort the features by their absolute values to see the most significant ones
    feature_importance_sorted = feature_importance.abs().sort_values(ascending=False)

    return feature_importance_sorted.head(20)

log_reg_important_features(log_reg)

<span style="color:cyan">

1. **Feature Coefficients Extraction**: It starts by obtaining the logistic regression model's coefficients, which quantify the impact of each feature on the model's prediction. A positive coefficient indicates a feature that increases the likelihood of the target variable being 1, while a negative coefficient suggests a feature that decreases this likelihood.

2. **Mapping Features to Coefficients**: The function maps these coefficients to the corresponding feature names from the `X_resampled` dataframe. This mapping is crucial for understanding which features are most influential in the model's decisions.

3. **Sorting by Importance**: The coefficients are then sorted by their absolute values in descending order. This sorting helps identify the features with the most significant impact on the model's predictions, regardless of the direction of their effect.

4. **Top 20 Features**: Finally, it returns the top 20 features with the highest absolute coefficient values, highlighting those with the greatest influence on the logistic regression model's outcomes.

The output reveals the features with the highest importance scores, notably including various `CRASH_TYPE` and `FIRST_CRASH_TYPE` categories, indicating the specific nature of a crash plays a crucial role in predicting its severity. Features like `PRIM_CONTRIBUTORY_CAUSE` related to the primary cause of the crash, and `NUM_UNITS` indicating the number of units involved in the crash, also appear as significant predictors. 

This insight into feature importance is invaluable for understanding the factors that most influence crash severity predictions, providing guidance for interventions and further research aimed at reducing severe traffic incidents.

</span>

In [ ]:
# 4.6. Modeling with DNN
implement_DNN(X_train, X_test, y_train, y_test)

<span style="color:cyan">

1. **DNN Implementation**: The `implement_DNN` function is called with the training and test datasets. This function encapsulates the entire process of configuring, training, and evaluating a DNN, including:
   - **Hyperparameter Tuning**: Utilizing a grid search to explore different combinations of `activation` functions and `hidden_layer_sizes` configurations. This systematic approach aims to identify the best set of parameters that result in the highest accuracy on the training data.
   - **Model Training**: With the optimal parameters identified (`{'activation': 'relu', 'hidden_layer_sizes': (10, 20)}`), the DNN is trained on the training dataset. The `relu` activation function and a two-layer neural network with 10 and 20 neurons, respectively, were found to be the best configuration.
   - **Evaluation on Test Data**: The trained DNN model is then evaluated on the unseen test dataset to assess its generalization capability.

2. **Model Performance**: The output reports the DNN model's performance metrics:
   - **Best Parameters**: Indicates the optimal combination of hyperparameters found through grid search.
   - **Accuracy**: The overall accuracy of the DNN model on the test set is approximately 84.45%, demonstrating a strong capability to correctly classify the severity of crashes.
   - **Classification Report**: Provides detailed performance metrics, including precision, recall, and F1-score for each severity class (`HIGH SEVERITY` and `LOW SEVERITY`). The model shows higher precision in predicting `LOW SEVERITY` crashes but exhibits a better recall for `HIGH SEVERITY` crashes, indicating it is more effective at identifying a larger proportion of high severity incidents, even though it's slightly more cautious in labeling crashes as low severity.

</span>


In [ ]:
# 4.7. Modeling with KNN
implement_KNN(X_train, X_val, X_test, y_train, y_val, y_test)


<span style="color:cyan">

Across the series of code blocks, we implemented and evaluated several machine learning models—Logistic Regression, Random Forest, Deep Neural Network (DNN), and K-Nearest Neighbors (KNN)—to predict crash severity from a dataset. Here's a summary of the key findings and model performances:

1. **Random Forest** emerged as a robust model with the best performance, achieving an accuracy of approximately 86.72%. It demonstrated high precision in predicting both `HIGH SEVERITY` and `LOW SEVERITY` crashes.

2. **Logistic Regression** followed closely, with an optimal regularization strength `C=2` and an accuracy of around 86.43%. It showed excellent ability to identify `HIGH SEVERITY` incidents.

3. **DNN** found an optimal configuration with `activation='relu'` and `hidden_layer_sizes=(10, 20)`, reaching an accuracy of 84.45%. The model balanced precision and recall reasonably well across severity classes.

4. **KNN**, with the best `n_neighbors=12`, had a lower accuracy of about 79.79% compared to the other models. It performed better in recall for `HIGH SEVERITY` but had lower precision for `LOW SEVERITY`.

Additionally, feature importance analyses for both the Random Forest and Logistic Regression models highlighted the significance of `CRASH_TYPE` and `FIRST_CRASH_TYPE` variables, underscoring the impact of crash characteristics on severity predictions.

Overall, each model provided valuable insights into the factors influencing crash severity, with Random Forest and Logistic Regression standing out in terms of accuracy and interpretability. These findings could guide future efforts to improve road safety and prevent severe crashes.
</span>

<br>
<br>
<br>
<br>
<br>

# 6. CRISP-DM: Deployment


## Deployment Strategy
We plan to make the model accessible via an API, hosted on a scalable cloud platform for reliability and flexibility. Continuous monitoring will ensure the model remains accurate and performs well over time.

## Integration Points
The model will seamlessly integrate into existing systems or applications, with a user interface potentially enhanced by interactive maps like Google Maps. This feature will provide end-users with intuitive, visual interpretations of the model's predictions, enriching the user experience.

## Maintenance Plan
To maintain model efficacy, we'll implement a schedule for regular updates and retraining with new data. Performance tuning, based on ongoing feedback and evolving data trends, will further refine the model.

## Security Measures
Ensuring data privacy and compliance with regulations is paramount. We'll implement robust access control measures to define clear permissions for interacting with the model, safeguarding user data.

## Success Metrics
To evaluate the deployment's success, we will track usage metrics to understand adoption rates and the frequency of use. An impact assessment will measure the tangible benefits and outcomes of the model's predictions in the real world.

<br>
<br>
<br>
<br>
<br>


# 7. Appendix - Extra Functionality & Bonus

7.1. Minimal vs. Expected vs. Bonus Functionality
7.2. Insights and Beyond

## 7.1. Minimal vs. Expected vs. Bonus Functionality

In this section we will detail what we have committed at the beginning of the project.

<br>

**Minimal Functionality**
* Data Collection and Preparation: Basic description and/or cleaning.
* Data Analysis: Basic analysis.
* Modeling: At least 1 algorith (e.g. Logistic Regression) + Evaluation + Resulst and Discussion.

<br>

**Expected Functionality**
* Data Collection and Preparation: Full description and cleaning.
* Data Analysis: Full analysis.
* Modeling: 2-3 models (LR, RF and KNN) + Evaluation + Resulst and Discussion.

<br>

**Bonus Functionality**
* Data Collection and Preparation: Advanced. We applied every technique learned during the course.
* Data Analysis: Advanced analysis + Insights (check details below).
* Modeling: We used 4 algorithms (LR, RF, KNN and DNN).
* Imbalanced Data & Sampling: For each model, we comparing results with sampling and without.
* Business Understanding: Domain Concepts and Full features details. It might be high investment, but was great to understand the data.

<br>

**Additional Comments**
1. Our code highly organized to functions and modern programing paradigms.
2. Well documented.
3. Analysis related part with "cyan" color to be more user-friendly to read.


<br>
<br>
<br>


<br>
<br>
<br>
<br>

## 7.2. Insights and Beyond

In this section, we present different insights and patterns that can be extracted from the dataset.

In [ ]:
df_for_insights = df_cleaned


In [ ]:
# Descriptive statistics for injury and fatality-related features
df_for_insights[['INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION']].describe()


In [ ]:
# Count occurrences of each severity level
print(df_for_insights['INJURIES_FATAL'].value_counts())
print(df_for_insights['INJURIES_INCAPACITATING'].value_counts())
print(df_for_insights['INJURIES_NON_INCAPACITATING'].value_counts())
print(df_for_insights['INJURIES_REPORTED_NOT_EVIDENT'].value_counts())
print(df_for_insights['INJURIES_NO_INDICATION'].value_counts())

# To get idea about the data ...

In [ ]:


def calculate_ratio_between_two_classes(my_df, feature_1, feature_2):
    # Calculate the total sum for each injury category
    total_feature_1 = my_df[feature_1].sum()
    total_feature_2 = my_df[feature_2].sum()
    # Calculate the ratio of incapacitating to fatal injuries
    ratio_feature_2_to_feature_1 = total_feature_2 / total_feature_1
    print(f"For every 1 {feature_1}, there are on average {ratio_feature_2_to_feature_1:.2f} {feature_2}")


print("Comparing Each Level to its next one:")
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_FATAL', 'INJURIES_INCAPACITATING')
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING')
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT')
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION')
print()
print("Comparing Each Level the base (lowest) level")
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_FATAL', 'INJURIES_NO_INDICATION')
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_INCAPACITATING', 'INJURIES_NO_INDICATION')
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_NON_INCAPACITATING', 'INJURIES_NO_INDICATION')
calculate_ratio_between_two_classes(df_for_insights, 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION')



In [ ]:
# Ratio of Non-Zero Injury Instance - Pie Chart
# Calculate the number of instances greater than zero for each injury type
injury_types = [
    'INJURIES_FATAL', 
    'INJURIES_INCAPACITATING', 
    'INJURIES_NON_INCAPACITATING', 
    'INJURIES_REPORTED_NOT_EVIDENT', 
    'INJURIES_NO_INDICATION'
]
injury_counts = {injury: (df_for_insights[injury] > 0).sum() for injury in injury_types}

# Create labels and sizes for the pie chart
labels = injury_counts.keys()
sizes = injury_counts.values()
colors = plt.cm.tab20c.colors  # Using a color map that has distinct colors for each category

# Plotting the pie chart
plt.figure(figsize=(10, 8))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie chart is drawn as a circle
plt.title('Ratio of Non-Zero Injury Instances')

# Show the pie chart
plt.show()


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))  # Adjust grid layout as needed
features = ['INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT']
titles = ['Fatal Injuries', 'Incapacitating Injuries', 'Non-Incapacitating Injuries', 'Reported Not Evident Injuries']

for ax, feature, title in zip(axes.flatten(), features, titles):
    ax.hist(df_for_insights[feature], bins=range(int(df_for_insights[feature].max() + 1)), color='skyblue', edgecolor='black')
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Number of ' + title, fontsize=14)
    ax.set_ylabel('Frequency', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(axis='y', alpha=0.75)

plt.tight_layout()  # Adjust subplots to fit into the figure area.
plt.show()


In [ ]:
# Analyzing injuries by month

# df_for_insights['CRASH_MONTH'] = pd.to_datetime(df_for_insights['CRASH_DATE']).dt.month
monthly_injuries = df.groupby('CRASH_MONTH')['INJURIES_TOTAL'].sum()
plt.plot(monthly_injuries.index, monthly_injuries.values)
plt.title('Total Injuries by Month')
plt.xlabel('Month')
plt.ylabel('Total Injuries')
plt.xticks(range(1, 13))
plt.show()


In [ ]:
# Total Injuries by Month (2018 - 2023)

# Filter data to exclude years 2015 and 2016, and include up to 2023
filtered_df = df_for_insights[(df_for_insights['YEAR'] > 2017) & (df_for_insights['YEAR'] <= 2023)]

# Group by year and month, summing total injuries
injuries_by_month_year = filtered_df.groupby(['YEAR', 'CRASH_MONTH'])['INJURIES_TOTAL'].sum().reset_index()

# Create a line plot
plt.figure(figsize=(12, 8))
line = sns.lineplot(data=injuries_by_month_year, x='CRASH_MONTH', y='INJURIES_TOTAL', hue='YEAR', palette='Spectral', marker='o', linewidth=2.5)

# Enhance the plot
plt.title('Total Injuries by Month (2018-2023)', fontsize=16)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Total Injuries', fontsize=14)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title='Year', title_fontsize='13', fontsize='12', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.grid(True)
plt.tight_layout()  # Adjust layout to make room for the legend
plt.show()


In [ ]:
# Fatal Traffic Accidents by Day of the Week

fatalities_per_day = df_for_insights.groupby('CRASH_DAY_OF_WEEK')['INJURIES_FATAL'].sum()

# Days of the week labels
days_of_the_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Creating the bar plot
plt.figure(figsize=(10, 6))
plt.bar(days_of_the_week, fatalities_per_day, color='tomato')
plt.title('Fatal Traffic Accidents by Day of the Week', fontsize=16)
plt.xlabel('Day of the Week', fontsize=14)
plt.ylabel('Number of Fatalities', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.show()


In [ ]:
# Fatal Traffic Accidents by Hour of the Day

# Grouping the data by 'CRASH_HOUR' and summing 'INJURIES_FATAL' to get total fatalities per hour
fatalities_per_hour = df_for_insights.groupby('CRASH_HOUR')['INJURIES_FATAL'].sum()

# Creating the bar plot
plt.figure(figsize=(12, 6))
plt.bar(fatalities_per_hour.index, fatalities_per_hour.values, color='royalblue', edgecolor='black')

# Adding labels and title
plt.title('Fatal Traffic Accidents by Hour of the Day', fontsize=16)
plt.xlabel('Hour of the Day (0-23)', fontsize=14)
plt.ylabel('Number of Fatalities', fontsize=14)

# Ensuring the x-axis covers all hours
plt.xticks(range(0, 24), fontsize=12)
plt.yticks(fontsize=12)

plt.grid(axis='y', alpha=0.75)
plt.show()


In [ ]:
# Fatal and Incapacitating Injuries by Hour of the Day


grouped_data = df_for_insights.groupby('CRASH_HOUR')[['INJURIES_FATAL', 'INJURIES_INCAPACITATING']].sum().reset_index()

# Setting the positions and width for the bars
pos = list(range(len(grouped_data['CRASH_HOUR'])))
width = 0.4

fig, ax = plt.subplots(figsize=(14, 8))

plt.bar(pos, 
        grouped_data['INJURIES_FATAL'], 
        width, 
        alpha=0.7, 
        color='red', 
        label='Fatal Injuries')

plt.bar([p + width for p in pos], 
        grouped_data['INJURIES_INCAPACITATING'],
        width, 
        alpha=0.7, 
        color='blue', 
        label='Incapacitating Injuries')

# Set the y-axis label
ax.set_ylabel('Number of Injuries')

# Set the chart's title
ax.set_title('Fatal and Incapacitating Injuries by Hour of the Day')

# Set the position of the x ticks
ax.set_xticks([p + width / 2 for p in pos])

# Set the labels for the x ticks
ax.set_xticklabels(grouped_data['CRASH_HOUR'])

# Setting the x-axis and y-axis limits
plt.xlim(min(pos)-width, max(pos)+width*2)
plt.ylim([0, max(grouped_data['INJURIES_FATAL'].max(), grouped_data['INJURIES_INCAPACITATING'].max())] )

# Adding the legend and showing the plot
plt.legend(['Fatal Injuries', 'Incapacitating Injuries'], loc='upper left')
plt.grid()
plt.show()


In [ ]:
# Debug Only - Helper functions to understand the data better


# dump_feature_frequency_to_a_file(df_to_model, 'PRIM_CONTRIBUTORY_CAUSE')

# dump_feature_classes_to_a_file(df_to_model, 'PRIM_CONTRIBUTORY_CAUSE')
# dump_feature_classes_to_a_file(df_to_model, 'SEC_CONTRIBUTORY_CAUSE')


# dump_feature_classes_to_a_file(df_to_model, 'TRAFFIC_CONTROL_DEVICE')
# dump_feature_classes_to_a_file(df_to_model, 'WEATHER_CONDITION')
